In [1]:
!pip install pickle5
!pip install transformers

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import os
import random
import pandas as pd
from torchtext.legacy import data
import pickle5 as pickle
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import params
import net
import numpy as np
from dataset import AmazonDataset
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
dataset = AmazonDataset() # fare prova con anche dataset non caricato
dataset.load_dataset()

Problemi:
1. Stiamo passando alla rete le frasi di testo (dataset.data) non tensori. In che formato vanno passati al modello? getitem restituisce una lista di elementi, dove ogni elemento è un tensore (ogni tensore corrisponde ad una parola della recenzione e ha lunghezza 3072) e la lunghezza della recensione è variabile
2. Attention weight va ancora chiamato
3. In dataset.embedded_words_dict ci sono parole (keys) con "##" davanti, credo non ci debbano essere? (vedi cella sotto)

In [9]:
# ci sono parole che hanno "##" davanti, credo vadano eliminate
print(dataset.embedded_words_dict['##o'])

tensor([-0.5865,  0.0887,  0.5736,  ..., -1.1254, -0.1932, -0.4799])


In [10]:
print(dataset.data[0])
print(dataset.__getitem__(0))
print(dataset.labels[0])



much write exactly supposed filters pop sounds recordings much crisp one lowest prices pop filters amazon might well buy honestly work despite pricing
[tensor([-0.0059, -0.0130,  0.1045,  ..., -0.0246,  0.0263, -0.0228]), tensor([0., 0., 0.,  ..., 0., 0., -0.]), tensor([ 0.0376,  0.0347,  0.0497,  ..., -0.0115,  0.0188,  0.0773]), tensor([0., -0., 0.,  ..., -0., 0., 0.]), tensor([0., 0., 0.,  ..., -0., -0., 0.]), tensor([0., 0., 0.,  ..., -0., -0., -0.]), tensor([-0., 0., 0.,  ..., -0., 0., -0.]), tensor([0., 0., 0.,  ..., -0., 0., -0.]), tensor([-0.0059, -0.0130,  0.1045,  ..., -0.0246,  0.0263, -0.0228]), tensor([-0., 0., 0.,  ..., -0., -0., 0.]), tensor([-0., 0., 0.,  ..., -0., 0., 0.]), tensor([-0.4410,  0.1086,  0.3828,  ..., -0.4357,  0.1334, -0.2306]), tensor([-0., 0., 0.,  ..., 0., 0., 0.]), tensor([0., 0., 0.,  ..., -0., -0., -0.]), tensor([0., 0., 0.,  ..., -0., -0., 0.]), tensor([-0.0177, -0.0042,  0.0537,  ...,  0.0291,  0.0579, -0.0338]), tensor([-0.1171,  0.0097,  0.0438,

In [11]:
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.labels, train_size=0.7, random_state=0, shuffle = True)

train_data = { 'features': x_train, 'labels': y_train}
test_data = {'features': x_test, 'labels': y_test}

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)


In [12]:
# Define parameters
batch_size = params.BATCH_SIZE
hidden_dim = 128
embedding_size = len(dataset.__getitem__(0)[0]) #lunghezza embedding (selezionato uno a caso perchè tanto tutti hanno la stessa dimensione)
dropout_rate = params.DROPOUT_RATE
lr = params.LR
epochs = params.NUM_EPOCHS
best_loss = float('inf')

# BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
# x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, train_size=0.8, random_state=0)
train_iterator, test_iterator = data.BucketIterator.splits((train_data, test_data),
    batch_size = batch_size,
    sort_key = lambda x: len(x.text), # Sort the batches by text length size
    sort_within_batch = True,
    device = device)

# Build the model
lstm_model = net.SentimentAnalysis(batch_size,
                                   hidden_dim,
                                   embedding_size,
                                   dropout_rate) #modificato LSTM con SentimentAnalysis (nome rete)

# optimization algorithm
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

In [13]:
to_train = True

# train and validate
if (to_train):

    for epoch in range(epochs):

        # training 
        epoch_loss = 0
        epoch_acc = 0
        for idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            # retrieve text and no. of words
            #text, text_lengths = batch.text
            
            predictions = lstm_model(batch)    #(text, text_lengths) # batch_size, hidden_dim, vocab_size, window, dropout_rate
            loss = 0.5*nn.CrossEntropyLoss(predictions, batch.labels.squeeze())+ 0.5*nn.MSELoss(predictions, batch.labels.squeeze())

            winners = predictions.argmax(dim=1)
            corrects = (winners == batch.labels)
            accuracy = corrects.sum().float() / float(batch.labels.size(0))

            # perform backpropagation
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += accuracy.item()
            
        train_loss, train_acc = epoch_loss / len(train_iterator), epoch_acc / len(train_iterator)

        # save best model
        if train_loss < best_loss:
            best_valid_loss = train_loss
            torch.save(lstm_model.state_dict(), 'saved_weights_BiLSTM.pt')

        print(f'\tEpoch; {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')  

KeyError: ignored

In [ ]:
# load weights and make predictions
lstm_model.load_state_dict(torch.load("saved_weights_BiLSTM.pt"))
epoch_loss = 0
epoch_acc = 0

lstm_model.eval()

with torch.no_grad():
    for batch in test_iterator:
        text, text_lengths = batch.text

        predictions = lstm_model(batch).squeeze(1)

        loss = 0.5*nn.CrossEntropyLoss(predictions, batch.labels)+ 0.5*nn.MSELoss(predictions, batch.labels)

        winners = predictions.argmax(dim=1)
        corrects = (winners == batch.labels)
        accuracy = corrects.sum().float() / float(batch.labels.size(0))

        epoch_loss += loss.item()
        epoch_acc += accuracy.item()

test_loss, test_acc =  epoch_loss / len(test_iterator), epoch_acc / len(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')